# 4 - TRABAJANDO CON BLPs

In [1]:
!pip install dwave-ocean-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.5/165.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00


In [2]:
import dimod
x0 = dimod.Binary("x0")
x1 = dimod.Binary("x1")
x2 = dimod.Binary("x2")

In [3]:
blp = dimod.ConstrainedQuadraticModel()
blp.set_objective(-5*x0+3*x1-2*x2)
blp.add_constraint(x0 + x2 <= 1, "Restricción 1")
blp.add_constraint(3*x0 -x1 + 3*x2 <= 4, "Restricción 2")

'Restricción 2'

In [4]:
print("Nuestras variables son:")
print(blp.variables)
print("Nuestra función objetivo es:")
print(blp.objective)
print("Nuestras restricciones son:")
print(blp.constraints)

Nuestras variables son:
Variables(['x0', 'x1', 'x2'])
Nuestra función objetivo es:
ObjectiveView({'x0': -5.0, 'x1': 3.0, 'x2': -2.0}, {}, 0.0, {'x0': 'BINARY', 'x1': 'BINARY', 'x2': 'BINARY'})
Nuestras restricciones son:
{'Restricción 1': Le(ConstraintView({'x0': 1.0, 'x2': 1.0}, {}, 0.0, {'x0': 'BINARY', 'x2': 'BINARY'}), np.float64(1.0)), 'Restricción 2': Le(ConstraintView({'x0': 3.0, 'x1': -1.0, 'x2': 3.0}, {}, 0.0, {'x0': 'BINARY', 'x1': 'BINARY', 'x2': 'BINARY'}), np.float64(4.0))}


In [6]:
sample1 = {"x0":1, "x1":1, "x2":1}
print("La asignación es", sample1)
print("Su coste es", blp.objective.energy(sample1))
print("¿Es factible?",blp.check_feasible(sample1))
print("Las restricciones no cumplidas son")
print(blp.violations(sample1))

La asignación es {'x0': 1, 'x1': 1, 'x2': 1}
Su coste es -4.0
¿Es factible? False
Las restricciones no cumplidas son
{'Restricción 1': np.float64(1.0), 'Restricción 2': np.float64(1.0)}


In [7]:
sample2 = {"x0":0, "x1":0, "x2":1}
print("La asignación es", sample2)
print("Su coste es", blp.objective.energy(sample2))
print("¿Es factible?",blp.check_feasible(sample2))
print("Las restricciones no cumplidas son")
print(blp.violations(sample2))

La asignación es {'x0': 0, 'x1': 0, 'x2': 1}
Su coste es -2.0
¿Es factible? True
Las restricciones no cumplidas son
{'Restricción 1': np.float64(0.0), 'Restricción 2': np.float64(-1.0)}


In [8]:
solver = dimod.ExactCQMSolver()
solution = solver.sample_cqm(blp)
print("La lista de asignaciones es:")
print(solution)

La lista de asignaciones es:
  x0 x1 x2 energy num_oc. is_sat. is_fea.
6  1  0  1   -7.0       1 arra... np.F...
2  1  0  0   -5.0       1 arra... np.T...
7  1  1  1   -4.0       1 arra... np.F...
3  1  1  0   -2.0       1 arra... np.T...
4  0  0  1   -2.0       1 arra... np.T...
0  0  0  0    0.0       1 arra... np.T...
5  0  1  1    1.0       1 arra... np.T...
1  0  1  0    3.0       1 arra... np.T...
['INTEGER', 8 rows, 8 samples, 3 variables]


In [9]:
feasible_sols = solution.filter(lambda s: s.is_feasible)
feasible_sols.first

Sample(sample={'x0': np.int64(1), 'x1': np.int64(0), 'x2': np.int64(0)}, energy=np.float64(-5.0), num_occurrences=np.int64(1), is_satisfied=array([ True,  True]), is_feasible=np.True_)

## Convirtiendo de BLP a QUBO"

In [10]:
y0, y1 = dimod.Binaries(["y0", "y1"])
cqm = dimod.ConstrainedQuadraticModel()
cqm.set_objective(-2*y0-3*y1)
cqm.add_constraint(y0 + 2*y1 <= 2)

'ccb8b22'

In [11]:
qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = 5)
print(qubo)

BinaryQuadraticModel({'y0': -17.0, 'y1': -23.0, 'slack_vb3454d4d5c844321bddcece16ee1930b_0': -15.0, 'slack_vb3454d4d5c844321bddcece16ee1930b_1': -15.0}, {('y1', 'y0'): 20.0, ('slack_vb3454d4d5c844321bddcece16ee1930b_0', 'y0'): 10.0, ('slack_vb3454d4d5c844321bddcece16ee1930b_0', 'y1'): 20.0, ('slack_vb3454d4d5c844321bddcece16ee1930b_1', 'y0'): 10.0, ('slack_vb3454d4d5c844321bddcece16ee1930b_1', 'y1'): 20.0, ('slack_vb3454d4d5c844321bddcece16ee1930b_1', 'slack_vb3454d4d5c844321bddcece16ee1930b_0'): 10.0}, 20.0, 'BINARY')


In [12]:
solver = dimod.ExactSolver()
result = solver.sample(qubo)
print("Las soluciones son")
print(result)

Las soluciones son
   slack_vb3454d4d5c844321bddcece16ee1930b_0 ... y1 energy num_oc.
3                                          0 ...  1   -3.0       1
6                                          1 ...  0   -2.0       1
14                                         0 ...  0   -2.0       1
2                                          0 ...  1    0.0       1
8                                          1 ...  0    0.0       1
4                                          1 ...  1    2.0       1
12                                         0 ...  1    2.0       1
1                                          0 ...  0    3.0       1
9                                          1 ...  0    3.0       1
7                                          1 ...  0    5.0       1
15                                         0 ...  0    5.0       1
5                                          1 ...  1   15.0       1
13                                         0 ...  1   15.0       1
11                                         

In [14]:
samples = []
occurrences = []
for s in result.data():
      samples.append(invert(s.sample))
occurrences.append(s.num_occurrences)
sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=occurrences)
print("Las soluciones del problema original son")
print(sampleset)

Las soluciones del problema original son
   y0 y1 energy num_oc. is_sat. is_fea.
3   1  1   -5.0       1 arra... np.F...
11  1  1   -5.0       1 arra... np.F...
12  1  1   -5.0       1 arra... np.F...
15  1  1   -5.0       1 arra... np.F...
0   0  1   -3.0       1 arra... np.T...
5   0  1   -3.0       1 arra... np.T...
6   0  1   -3.0       1 arra... np.T...
13  0  1   -3.0       1 arra... np.T...
1   1  0   -2.0       1 arra... np.T...
2   1  0   -2.0       1 arra... np.T...
7   1  0   -2.0       1 arra... np.T...
8   1  0   -2.0       1 arra... np.T...
4   0  0    0.0       1 arra... np.T...
9   0  0    0.0       1 arra... np.T...
10  0  0    0.0       1 arra... np.T...
14  0  0    0.0       1 arra... np.T...
['INTEGER', 16 rows, 16 samples, 2 variables]


In [15]:
final_sols = sampleset.filter(lambda s: s.is_feasible)
final_sols = final_sols.aggregate()
print("Las soluciones finales son")
print(final_sols)

Las soluciones finales son
  y0 y1 energy num_oc. is_sat. is_fea.
0  0  1   -3.0       4 arra... np.T...
1  1  0   -2.0       4 arra... np.T...
2  0  0    0.0       4 arra... np.T...
['INTEGER', 3 rows, 12 samples, 2 variables]
